<a href="https://colab.research.google.com/github/anshidungani/shapeaiproject/blob/main/python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install keras-tuner

     |████████████████████████████████| 71kB 2.0MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp37-none-any.whl size=78938 sha256=ab47206fdd2d2b03f3121146733a76ab6981e68370399e62e414c6341e1fb082
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp37-none-any.whl size=15356 sha256=4fb72ca43bc785b6b692773d13fb71c1ea3be1958d2173de59116ff797e600ad
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [ ]:
fashion_mnist=keras.datasets.fashion_mnist

In [ ]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [ ]:
train_images=train_images/255.0
test_images=test_images/255.0

In [ ]:
train_images[0].shape

(28, 28)

In [ ]:
train_images = train_images.reshape(len(train_images),28,28,1)
test_images = test_images.reshape(len(test_images),28,28,1)

In [ ]:
def build_model(hp):
  model=keras.Sequential([
                          keras.layers.Conv2D(
                              filters=hp.Int('conv_1_filter',min_value=32,max_value=128,step=16),
                              kernel_size=hp.Choice('conv_1_kernel',values=[3,5]),
                              activation='relu',
                              input_shape=(28,28,1)
                          ),
                          keras.layers.Conv2D(
                              filters=hp.Int('conv_2_filter',min_value=32,max_value=64,step=16),
                              kernel_size=hp.Choice('conv_2_kernel',values=[3,5]),
                              activation='relu'
                          ),
                          keras.layers.Flatten(),
                          keras.layers.Dense(
                              units=hp.Int('dense_1_units',min_value=32,max_value=128,step=16),
                              activation='relu'
                          ),
                          keras.layers.Dense(10,activation='softmax')
  ])
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

  return model

In [ ]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [ ]:
tuner_search = RandomSearch(build_model, objective='val_accuracy', max_trials=5, directory='output', project_name="Mnist Fashion")

In [ ]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 7 Complete [00h 06m 22s]
val_accuracy: 0.9066666960716248

Best val_accuracy So Far: 0.9098333120346069
Total elapsed time: 01h 00m 47s

Search: Running Trial #8

Hyperparameter    |Value             |Best Value So Far 
conv_1_filter     |64                |112               
conv_1_kernel     |3                 |3                 
conv_2_filter     |64                |64                
conv_2_kernel     |5                 |3                 
dense_1_units     |96                |112               
learning_rate     |0.01              |0.001             

Epoch 1/3
1688/1688 [==============================] - 462s 274ms/step - loss: 0.5313 - accuracy: 0.8227 - val_loss: 0.4432 - val_accuracy: 0.8377
Epoch 2/3
  27/1688 [..............................] - ETA: 7:28 - loss: 0.3752 - accuracy: 0.8657

In [17]:
model=tuner_search.get_best_models(num_models=1)[0]

In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 112)       1120      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 64)        64576     
_________________________________________________________________
flatten (Flatten)            (None, 36864)             0         
_________________________________________________________________
dense (Dense)                (None, 112)               4128880   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1130      
Total params: 4,195,706
Trainable params: 4,195,706
Non-trainable params: 0
_________________________________________________________________
